CSV -> db 변환 (sql 사용을 위함)

In [6]:
import pandas as pd
from sqlalchemy import create_engine

# CSV 경로 (EDA.ipynb 기준 상대경로)
csv_path = "../datasets/PaySim.csv"

# CSV 로드
df = pd.read_csv(csv_path)

# SQLite DB 생성/연결 (local DB 파일 생성됨)
engine = create_engine("sqlite:///paysim.db")

# PaySim이라는 테이블명으로 저장
df.to_sql("PaySim", engine, if_exists="replace", index=False)

print("테이블 생성 완료!")


테이블 생성 완료!


In [11]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%sql sqlite:///paysim.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


---

### **✔️  1. 기본 구조 확인**

1) 컬럼 정보 확인

In [22]:
%%sql
PRAGMA table_info(PaySim);

 * sqlite:///paysim.db
   sqlite:///test.db
Done.


cid,name,type,notnull,dflt_value,pk
0,step,BIGINT,0,None,0
1,type,TEXT,0,None,0
2,amount,FLOAT,0,None,0
3,nameOrig,TEXT,0,None,0
4,oldbalanceOrg,FLOAT,0,None,0
5,newbalanceOrig,FLOAT,0,None,0
6,nameDest,TEXT,0,None,0
7,oldbalanceDest,FLOAT,0,None,0
8,newbalanceDest,FLOAT,0,None,0
9,isFraud,BIGINT,0,None,0


2) 전체 행 개수 확인

In [28]:
%%sql
SELECT COUNT(*) AS total_rows
FROM PaySim;



 * sqlite:///paysim.db
   sqlite:///test.db
Done.


total_rows
114957


3) 숫자형 컬럼들의 기본 통계 요약

In [30]:
%%sql
SELECT
    MIN(amount) AS min_amount,
    MAX(amount) AS max_amount,
    AVG(amount) AS avg_amount,
    SUM(amount) AS total_amount,
    MIN(oldbalanceOrg) AS min_oldOrg,
    MAX(oldbalanceOrg) AS max_oldOrg,
    AVG(oldbalanceOrg) AS avg_oldOrg
FROM PaySim;



 * sqlite:///paysim.db
   sqlite:///test.db
Done.


min_amount,max_amount,avg_amount,total_amount,min_oldOrg,max_oldOrg,avg_oldOrg
0.92,36973901.85,180774.60720133607,20781306520.04399,0.0,33593209.9,844075.735727881


4) 고유값 조회

In [27]:
%%sql
SELECT DISTINCT type FROM PaySim;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


type
CASH_IN
PAYMENT
TRANSFER
CASH_OUT
DEBIT


---

### **✔️  2. 결측치 / 데이터 품질 점검**

5) 컬럼별 결측치 개수

In [31]:
%%sql
SELECT
    SUM(CASE WHEN step IS NULL THEN 1 ELSE 0 END) AS step_null,
    SUM(CASE WHEN type IS NULL THEN 1 ELSE 0 END) AS type_null,
    SUM(CASE WHEN amount IS NULL THEN 1 ELSE 0 END) AS amount_null,
    SUM(CASE WHEN nameOrig IS NULL THEN 1 ELSE 0 END) AS nameOrig_null,
    SUM(CASE WHEN oldbalanceOrg IS NULL THEN 1 ELSE 0 END) AS oldbalanceOrg_null,
    SUM(CASE WHEN newbalanceOrig IS NULL THEN 1 ELSE 0 END) AS newbalanceOrig_null,
    SUM(CASE WHEN nameDest IS NULL THEN 1 ELSE 0 END) AS nameDest_null,
    SUM(CASE WHEN oldbalanceDest IS NULL THEN 1 ELSE 0 END) AS oldbalanceDest_null,
    SUM(CASE WHEN newbalanceDest IS NULL THEN 1 ELSE 0 END) AS newbalanceDest_null,
    SUM(CASE WHEN isFraud IS NULL THEN 1 ELSE 0 END) AS isFraud_null,
    SUM(CASE WHEN isFlaggedFraud IS NULL THEN 1 ELSE 0 END) AS isFlaggedFraud_null,
    SUM(CASE WHEN signup_step IS NULL THEN 1 ELSE 0 END) AS signup_step_null,
    SUM(CASE WHEN isActivation IS NULL THEN 1 ELSE 0 END) AS isActivation_null,
    SUM(CASE WHEN isReferral IS NULL THEN 1 ELSE 0 END) AS isReferral_null
FROM PaySim;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


step_null,type_null,amount_null,nameOrig_null,oldbalanceOrg_null,newbalanceOrig_null,nameDest_null,oldbalanceDest_null,newbalanceDest_null,isFraud_null,isFlaggedFraud_null,signup_step_null,isActivation_null,isReferral_null
0,0,0,0,0,0,0,0,0,0,0,0,0,0


5) 중복 행 개수

SQLite에서는 아래처럼 전체 행 중복을 직접 비교하는 기능이 없음
-> 문자열을 이어 붙이는(concatenation) 연산자를 사용해 여러 컬럼을 하나로 붙여서 각 row를 하나의 문자열 key처럼 취급
-> 중복된 문자열 key 개수를 세는 방식

In [32]:
%%sql
SELECT COUNT(*) - COUNT(DISTINCT step || type || amount || nameOrig || nameDest) 
AS approx_duplicate_rows
FROM PaySim;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


approx_duplicate_rows
0


---

### **✔️  3. Fraud 데이터 분석에 필요한 기본 숫자/관계**

7) Fraud 비율

In [33]:
%%sql
SELECT
    isFraud,
    COUNT(*) AS cnt,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM PaySim), 2) AS ratio
FROM PaySim
GROUP BY isFraud;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


isFraud,cnt,ratio
0,114786,99.85
1,171,0.15


8) Flagged Fraud 비율

In [34]:
%%sql
SELECT
    isFlaggedFraud,
    COUNT(*) AS cnt
FROM PaySim
GROUP BY isFlaggedFraud;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


isFlaggedFraud,cnt
0,114956
1,1


9) Fraud 트랜잭션에서 amount 평균/최대/합계

In [35]:
%%sql
SELECT
    AVG(amount) AS avg_fraud_amount,
    MAX(amount) AS max_fraud_amount,
    SUM(amount) AS total_fraud_amount
FROM PaySim
WHERE isFraud = 1;


 * sqlite:///paysim.db
   sqlite:///test.db
Done.


avg_fraud_amount,max_fraud_amount,total_fraud_amount
1272081.2662097088,10000000.0,217525896.52186018


10) Fraud 거래의 잔액 변화 특징

In [37]:
%%sql
SELECT
    AVG(oldbalanceOrg) AS avg_oldbalance,
    AVG(newbalanceOrg) AS avg_newbalance,
    AVG(oldbalanceDest) AS avg_dest_old,
    AVG(newbalanceDest) AS avg_dest_new
FROM PaySim
WHERE isFraud = 1;


 * sqlite:///paysim.db
   sqlite:///test.db
(sqlite3.OperationalError) no such column: newbalanceOrg
[SQL: SELECT
    AVG(oldbalanceOrg) AS avg_oldbalance,
    AVG(newbalanceOrg) AS avg_newbalance,
    AVG(oldbalanceDest) AS avg_dest_old,
    AVG(newbalanceDest) AS avg_dest_new
FROM PaySim
WHERE isFraud = 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


---